In [1]:
import string
import numpy as np

In [51]:
m = 29
alphabet = string.ascii_lowercase + "äöü"
alphabet

'abcdefghijklmnopqrstuvwxyzäöü'

In [53]:
def convert_char_to_indcies(elements):
    result = []
    for element in elements:
        current_indices = []
        for char in element:
            index = alphabet.find(char)
            current_indices.append(index)
        result.append(current_indices)
    return result

def get_word_diff_vector(w0, elements):
    result = []
    for i in range(len(elements)):
        w0i = w0[i]
        element = elements[i]
        diff = (element - w0i) % m
        result.append(diff)
    return result

def convert_indices_to_string(indices):
    result = ""
    for index in indices:
        current_char = alphabet[index]
        result += current_char
    return result

In [4]:
words = ["nase", "hase", "wick", "wack", "fupp"]
chiffres = ["hezg", "hlzr", "lfbb", "nftb", "nkäf"]

In [54]:
word_indices = convert_char_to_indcies(words)
chiffre_indices = convert_char_to_indcies(chiffres)

word_indices

[[13, 0, 18, 4],
 [7, 0, 18, 4],
 [22, 8, 2, 10],
 [22, 0, 2, 10],
 [5, 20, 15, 15]]

In [108]:
chiffre_indices

[[7, 4, 25, 6],
 [7, 11, 25, 17],
 [11, 5, 1, 1],
 [13, 5, 19, 1],
 [13, 10, 26, 5]]

In [55]:
w0 = word_indices[0]
W = []
for word_index in word_indices[1:]:
    W.append(get_word_diff_vector(w0, word_index))

c0 = chiffre_indices[0]
C = []
for chiffre_index in chiffre_indices[1:]:
    C.append(get_word_diff_vector(c0, chiffre_index))

W

[[23, 0, 0, 0], [9, 8, 13, 6], [9, 0, 13, 6], [21, 20, 26, 11]]

In [109]:
C

[[0, 7, 0, 11], [4, 1, 5, 24], [6, 1, 23, 24], [6, 6, 1, 28]]

In [73]:
np_W = np.array(W).T
np_W

array([[23,  9,  9, 21],
       [ 0,  8,  0, 20],
       [ 0, 13, 13, 26],
       [ 0,  6,  6, 11]])

In [82]:
np_C = np.array(C).T
np_C

array([[ 0,  4,  6,  6],
       [ 7,  1,  1,  6],
       [ 0,  5, 23,  1],
       [11, 24, 24, 28]])

In [110]:
det_W = int(np.round(np.linalg.det(np_W) % m))
det_W

15

In [112]:
def minor_det(matrix, i, j):
    sub_matrix = np.delete(matrix, i-1, 0)
    sub_matrix = np.delete(sub_matrix, j-1, 1)
    det = np.linalg.det(sub_matrix)
    return np.round(det)


def get_adjugate(matrix):
    shape = matrix.shape
    cofactor_matrix = np.zeros(shape)
    for i in range(1, shape[0]+1):
        for j in range(1, shape[0]+1):
            cofactor_matrix[i-1][j-1] = ((-1)**(i+j)*minor_det(matrix,i,j))
    return cofactor_matrix.T
    


In [114]:
# Aus voriger Zelle klar
inv_det_W  = 2
adj = get_adjugate(np_W).astype(int) % m
adj

array([[12,  0, 13,  7],
       [ 0, 20,  5, 23],
       [ 0,  9, 18,  7],
       [ 0,  0, 27, 14]])

In [117]:
adjDet = (inv_det_W * adj) % m
A = np.matmul(np_C, adjDet) % m
A

array([[ 0,  7,  0,  1],
       [23,  0,  1,  7],
       [ 0,  5,  4,  0],
       [ 3,  0,  2,  0]])

In [118]:
b = (c0 - np.matmul(A, w0)) % m
b

array([ 3,  7, 11, 18])

In [104]:
k = (A,b)
k

(array([[ 0,  7,  0,  1],
        [23,  0,  1,  7],
        [ 0,  5,  4,  0],
        [ 3,  0,  2,  0]]),
 array([ 3,  7, 11, 18]))

In [119]:
chiffre = "vxfe"
chiffre_conv = convert_char_to_indcies(chiffre)
chiffre_conv = [value for sublist in chiffre_conv for value in sublist]
chiffre_conv


[21, 23, 5, 4]

# b)
Klartext = "vxfe"

- Inverse von A berechnen
- Entschlüsselungsfunktion mit $ A^{-1} $ anwenden

In [123]:
detA = int(np.round(np.linalg.det(A))) % m
detA

25

In [124]:
inv_det_A = 7
(detA * inv_det_A) % m

1

In [126]:
adjA = get_adjugate(A).astype(int) % m
adjA

array([[12, 19, 18, 27],
       [26, 17, 15,  5],
       [11, 15,  2,  1],
       [17, 26, 11, 23]])

In [129]:
invA = (inv_det_A * adjA) % m
invA


array([[26, 17, 10, 15],
       [ 8,  3, 18,  6],
       [19, 18, 14,  7],
       [ 3,  8, 19, 16]])

In [142]:
plain_text = np.matmul(invA, (chiffre_conv - b)) % m
plain_text = convert_indices_to_string(plain_text)
plain_text


'gans'